<a href="https://colab.research.google.com/github/Alerciosilva9/correcao-automatica/blob/main/CORRECAO_AUTOMATICA_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IMPORT BIBLIOTECAS**

In [ ]:
import spacy
!python -m spacy download pt_core_news_md
#!python -m spacy download pt_core_news_lg
#!python -m spacy download pt_core_news_sm



/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-01-11 01:30:40.197302: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 16.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_md')


In [ ]:
import csv
from google.colab import drive
from spacy.matcher import Matcher
drive.mount('/content/drive')
nlp = spacy.load('pt_core_news_md')
matcher = Matcher(vocab=nlp.vocab)

In [ ]:
text2 = 'A cidade possui alguns parques e algumas praias'


doc2 = nlp(text2)

for token in doc2:
  print(token.text)
  print(token.pos_)
  print(token.dep_)
  print(token.lemma_,'\n')
 

print('-------------------------\n\n')
text = 'O carro e casa foram comprados'

doc = nlp(text)

for token in doc:
  print(token.text)
  print(token.pos_)
  print(token.dep_)
  print(token.lemma_,'\n')


In [ ]:
!pip install cyhunspell


In [ ]:
import hunspell

h = hunspell.Hunspell()

print(h.lang)

# **FUNÇÕES**

In [ ]:
def vectortostring(vector):
  aux = ''
  for v in vector:
    aux = aux + ' ' + v

  return aux

def returnposs(text):
  dep = []
  doc = nlp(text)
  for token in doc:
    if((token.pos_)!='SPACE'):
      dep.append(token.pos_)

  return dep

def returnregras(text):
  dep = []
  doc = nlp(text)
  for token in doc:
    if((token.pos_)!='SPACE'):
      dep.append(token.dep_)

  return dep

def tratarconjuncao(texto):
  sentencas = []
  doc = nlp(texto)
  pos = []
  dep = []
  txts = []
  txt = ''
  #print('\nSENTENCA: ',texto)
  for token in doc:
    pos.append(token.pos_)
    dep.append(token.dep_)
    txts.append(token.text)
  

  if(len(pos)<=2):
    sentencas.append(texto)
    return sentencas
  
  if(',' in txts):
    sentencasvirg = []
    list = texto.split(',')
    for l in list:
      if((l.count(' '))<=2):
        sentencas.append(texto)
        return sentencas
      else:
        sentencasvirg.append([l])
    return sentencasvirg   
  
  if('CCONJ' in pos):
    aux = pos.index('CCONJ')
    if((len(pos)+1)-(aux)>=3):
      if(pos[aux+1] == 'DET' and txts[aux-1]!=':'):
        sentencas.append(txts[0:aux])
        sentencas.append(txts[aux+1:len(pos)])
        return sentencas
 

  
  
  sentencas.append(texto)
  return sentencas

def tratarsentencas(text):
  sents = []
  virg = []
  #doc = nlp(text)
  aux = []
  lista = text.split('.')
  for s in lista:
    if(len(s) > 2):
      aux = tratarconjuncao(s)
      if(len(aux)<=1):
        sents.append(aux[0])
      else:
        for t in aux:
          sents.append(vectortostring(t))
    
  return sents


def indextoword(sentenca,vec):
  doc = nlp(sentenca)
  txt = []
  dep = []
  pos = []
  #proibir ROOT,
  permission = ['amod','nsubj','obj']
  result = []
  for token in doc:
    #print('DEP ',token.dep_)
    txt.append(token.text)
    dep.append(token.dep_)
    pos.append(token.pos_)
  for x in vec:
    #print(pos[x])
    if(dep[x] in permission or pos[x] is 'NOUN'):
      result.append(txt[x])
  return result


def invector(vec1,vec2):
  for v in vec2:
    if (v in vec1):
      return True
  return False


def recentidade(s,sen):
  pos_sconj_noun = [{'POS': 'SCONJ'},{'POS': 'NOUN'}]
  det_amod = [{'DEP': 'det'},{'DEP': 'amod'}]
  det_nsubj = [{'DEP': 'det'},{'DEP': 'nsubj'}]
  matcher.add("det+nsubj", patterns=[det_nsubj])
  matcher.add("det+amod", patterns=[det_amod])
  matcher.add("pos+sconj+noun", patterns=[pos_sconj_noun])
  doc = nlp(s)
  dep = []
  for n in doc:
    dep.append(n.dep_)
  #print('\nDEP: ',dep)
  result = matcher(doc, as_spans=True)
  flag = False
  aux1 = []
  aux2 = []
  for r in result:
    aux2 = []
    falg = False
    for x in range(r.start, r.end+1):
      aux2.append(x)
    if(len(aux1)==0):
      aux1.append(aux2)
      #print(r.label_)
    else:
      for x in aux1:
        flag = invector(x,aux2)
        if(flag):
          break
      if(flag is False):
          aux1.append(aux2)
          #print(r.label_)

  ent = []
  return aux1


def padraoentidade(entidade,sentenca):
  texto = []
  regra = []
  txt = []
  dep = []
  doc = nlp(sentenca)

  if(len(entidade)==1):
    for token in doc:
      if((token.pos_)!='SPACE'):
        texto.append(token.text)
        if(token.text==entidade[0]):
          regra.append('ENTITY')
        else:
          regra.append(token.dep_)
  elif(len(entidade)==2):
    for token in doc:
      if((token.pos_)!='SPACE'):
        txt.append(token.text)
        dep.append(token.dep_)
    aux = 0
    for t in txt:
      if(entidade[0]==t ):
        texto.append(entidade[0] + ' ' + entidade[1])
        regra.append('ENTITY')
      elif(entidade[1] == t):
        #print('teste)
        aux =aux
      else:
        texto.append(t)
        regra.append(dep[aux])

      aux = aux + 1
  

  result = []
  result.append(texto)
  result.append(regra)
  result.append(entidade)
  return result

<>:102: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:102: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-1-a1d9379336fa>:102: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if(dep[x] in permission or pos[x] is 'NOUN'):


In [ ]:
def padraoenc(list, padrao):
  ind = []
  aux = 0
  final = []
  flag = 0
  for l in list:
    if(l==padrao[0]) :
      ind.append(aux)   
    aux = aux + 1
  boo = False
  for i in ind:
    boo = False
    flag = 1
    if((len(list)-i)>2):
      for p in padrao[1:(len(padrao))]:
        if(p==list[i+flag]):
          boo = True
        else:
          boo = False
        flag = flag+1
    if(boo):
      final.append(i)
    #print('I: ',i)
    #print('L: ',len(list))
    return final

def extrairprep(padrao):
  
  log = []
  aux=0
  reg = []
  txt = []
  cnt = 0
  for x in padrao[1]:
    reg.append(x)
    txt.append(padrao[0][cnt])
    cnt = cnt + 1
  #print(reg)
  #print(txt)
  #PADROES---------
  #print(padraoenc(reg,['ENTITY', 'ROOT', 'obj']))
  if(len(padraoenc(reg,['ENTITY', 'ROOT', 'obj']))>0):
    for z in padraoenc(reg,['ENTITY', 'ROOT', 'obj']):
      print(padrao[0][z]+'-->'+padrao[0][z+1]+'-->'+padrao[0][z+2])
  #elif():





  return log

# **CODE**

In [ ]:
with open("/content/drive/MyDrive/PYTHON/student_answers_and_grades_v2_other_graders.csv", 'r') as file:
  csvreader = csv.reader(file)
  count = 0
  padroes = []
  sentencas = []
  aux = []
  aux2 = []
  for row in csvreader:
    sentencas = []
    count = count + 1
    if(count<1000):
      #print('\n\n\n\n-----------------------------------------------')
      #print('\nSENTENCAS')
      sentencas = tratarsentencas(row[1])
      for s in sentencas:
        #print('S: ',s)
        #print('R: ',returnregras(s))
        #print('P: ',returnposs(s))
        aux = recentidade(s,sentencas)
        if(len(aux)>0):
          padroes = []
          print('\n-------------------------------------------------------')
          print('\n\nS: ',s)
          for x in aux:
            extrairprep(padraoentidade(indextoword(s,x),s))
            print('')
            '''aux2 = []
            for z in x:
              aux2.append(z)
            print('ENTIDADE: ', indextoword(s,aux2))'''
      #print(returnregras(row[1]))







NameError: ignored

# **ANALISE**

In [ ]:
#ANALISAR FUNCAO SINTATICA DE CADA TERMO
with open("/content/drive/MyDrive/PYTHON/reference_answers_extended.csv", 'r') as file:
  csvreader = csv.reader(file)
  count = 0
  padroes = []
  sentencas = []
  aux = []
  aux2 = []
  for row in csvreader:
    count = count + 1
    if(count<1000):
      if((row[0]) == '36'):
        print('\n\n-----------------------------------------------------------------------------------------------------------\nS: ',row[1])
        print('R: ',returnregras(row[1]))



-----------------------------------------------------------------------------------------------------------
S:  A célula animal possui formato irregular e a célula vegetal apresenta uma forma fixa. A célula vegetal possui parede celular, já que é uma exclusividade. A célula animal tem a ausência de vacúolo, plastos e Glioxissomos.
R:  ['det', 'nsubj', 'amod', 'ROOT', 'obj', 'amod', 'cc', 'det', 'nsubj', 'amod', 'conj', 'det', 'obj', 'amod', 'punct', 'det', 'nsubj', 'amod', 'ROOT', 'obj', 'amod', 'punct', 'mark', 'fixed', 'cop', 'det', 'advcl', 'punct', 'det', 'nsubj', 'amod', 'ROOT', 'det', 'obj', 'case', 'nmod', 'punct', 'conj', 'cc', 'conj', 'punct']


-----------------------------------------------------------------------------------------------------------
S:  Nos vegetais, a célula apresenta, além da membrana plasmática, uma parede que a reveste externamente. Formada principalmente por celulose, essa parede celular garante maior resistência à célula vegetal, evitando a ruptura p

In [ ]:
pip install pysinonimos

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 271 kB 4.7 MB/s 
     |████████████████████████████████| 4.1 MB 66.5 MB/s 
     |████████████████████████████████| 254 kB 66.4 MB/s 
     |████████████████████████████████| 3.1 MB 28.1 MB/s 
     |████████████████████████████████| 93 kB 2.6 MB/s 
     |████████████████████████████████| 57 kB 5.2 MB/s 
     |████████████████████████████████| 74 kB 3.5 MB/s 
  Created wheel for pysinonimos: filename=pysinonimos-0.1.1-py3-none-any.whl size=2451 sha256=f1b2772b4ff164689de44f6768392d4f7e63bd323aa1a74e3851273bdea7f181
  Stored in directory: /root/.cache/pip/wheels/cf/3b/df/dc55668d011a2c8fc001234634f3e35df76b4cc6de5287ca4f
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.6-py3-none-any.whl size=11959 sha256=4f06bbb375ca46fb89a6a845b4b7c18ce6f606e0fa317c4863f06fbe223a3db6
  Stored in directory: /root/.cache/pip/wheels/c9/d6/6a/de198d890277cde60c

In [ ]:
from pysinonimos.sinonimos import Search, historic

apresentacao = Search("tem")
sinonimos_de_apresentacao = apresentacao.synonyms()
print(sinonimos_de_apresentacao)



historico = historic()
#print(historico)

['possui', 'dispõe', 'detém', 'conta', 'há', 'traz', 'leva', 'porta', 'contém', 'inclui', 'abrange', 'compreende', 'apresenta', 'comporta', 'consta', 'existe', 'dura', 'passa', 'vive', 'experiencia', 'sofre', 'gera', 'pare', 'cria', 'produz', 'pega', 'apanha', 'fica', 'adquire', 'recebe', 'veste', 'guarda', 'conserva', 'retém', 'mantém', 'sustenta', 'segura', 'agarra', 'detém', 'para', 'precisa', 'necessita', 'carece', 'considera', 'toma', 'estima', 'revela', 'manifesta', 'demonstra', 'mostra', 'evidencia', 'expressa', 'diz', 'consegue', 'conquista', 'atinge', 'realiza', 'faz', 'efetiva', 'compra', 'adquire', 'fica', 'merca', 'contém', 'encontra', 'atende', 'transa', 'tem-se']


In [ ]:
#ANALISAR FUNCAO SINTATICA DE CADA TERMO

with open("/content/drive/MyDrive/PYTHON/student_answers_and_grades_v2.csv", 'r') as file:
  csvreader = csv.reader(file)
  count = 0
  padroes = []
  sentencas = []
  aux = []
  aux2 = []
  for row in csvreader:
    count = count + 1
    print(row[0])

  
  print('FINAL2: ',count)

A saída de streaming foi truncada nas últimas 5000 linhas.
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38

# **COMPARAÇÃO**

In [ ]:
#A célula animal e vegetal apresentam formato diferenciado. A célula animal possui formato irregular, enquanto a célula vegetal apresenta uma forma fixa.
#

In [ ]:
text = 'A membrana protege a célula e permite trocas '
dep = []
doc = nlp(text)
pos = []
head = []
for token in doc:
  if((token.pos_)!='SPACE'):
    dep.append(token.dep_)
    head.append(token.head)
    pos.append(token.pos_)
    

print(dep)
print(pos)
print(head)

['det', 'nsubj', 'ROOT', 'det', 'obj', 'cc', 'conj', 'obj']
['DET', 'NOUN', 'VERB', 'DET', 'NOUN', 'CCONJ', 'VERB', 'NOUN']
[membrana, protege, protege, célula, protege, permite, protege, permite]
